In [ ]:
import numpy as np
import matplotlib as mpl
import os

import scripts
from scripts import sbi_tools
from scripts import plot_utils

%load_ext autoreload
%autoreload 2

%matplotlib inline
mpl.pyplot.style.use('default')
mpl.pyplot.close('all')

font, rcnew = plot_utils.matplotlib_default_config()
mpl.rc('font', **font)
mpl.pyplot.rcParams.update(rcnew)
mpl.pyplot.style.use('tableau-colorblind10')
%config InlineBackend.figure_format = 'retina'

N_threads = sbi_tools.set_N_threads(6)

In [ ]:
path_script_dir = ".."

biases_vec = np.loadtxt(os.path.join(path_script_dir, "data", "biases_vec.txt"))

indicesLH = []
for ii, dir_name in enumerate(os.listdir(os.path.join(path_script_dir, "data/biased_pks"))):
    if dir_name.split('_')[0] == "biased":
        indicesLH.append(int(dir_name.split('_')[4]))
indicesLH = np.array(indicesLH)
indicesLH = np.unique(indicesLH)

#indicesLH = indicesLH[0:1]

fig, ax = mpl.pyplot.subplots(figsize=(8, 6))
Pk = []
for iLH in range(len(indicesLH)):
    for ib in range(len(biases_vec)):
        kk, tmp_pk = np.loadtxt(os.path.join(
            path_script_dir, "data/biased_pks", 'biased_pk_m2m_num_%04d_bias_num_%d.txt'%(indicesLH[iLH], ib)
        ), unpack=True)
        Pk.append(tmp_pk)
        mask = tmp_pk > 0
        ax.loglog(kk[mask], tmp_pk[mask], label='bias %d'%ib)
Pk = np.array(Pk)
print(Pk.shape)

ax.set_xlabel(r'$k \,\, [h \,\, {\rm Mpc}^{-1}]$', fontsize=23)
ax.set_ylabel(r'$P(k) \,\, [h^{-3} \,\, {\rm Mpc}^3]$', fontsize=23)

mpl.pyplot.tight_layout()
mpl.pyplot.show()

In [ ]:
path_read_theta = "/dipc_storage/mpelle/Yin_data/Quijote"

theta = []
for iLH, id_LH in enumerate(indicesLH):
    theta.append(np.loadtxt(
        os.path.join(
            path_read_theta, "LH" + str(id_LH).zfill(4), "param_"+str(id_LH).zfill(4)+".txt")
    ))
theta = np.array(theta)
theta = np.repeat(theta, 10, axis=0)

In [ ]:
n_biasmodels = len(biases_vec)
n_cosmos = int(theta.shape[0]/n_biasmodels)

In [ ]:
rng = np.random.default_rng(seed=533)
n_tot = len(Pk)

i_blocks = np.arange(n_tot).reshape(n_cosmos, n_biasmodels)

i_shuffle_blocks = rng.choice(np.arange(n_cosmos), replace=False, size=n_cosmos)

i_shuffle = i_blocks[i_shuffle_blocks].flatten()

Pk = Pk[i_shuffle]
theta = theta[i_shuffle]


In [ ]:
train_val_split = int(n_biasmodels*round(theta.shape[0]*0.99/n_biasmodels))

theta_train = theta[:train_val_split]
theta_test = theta[train_val_split:]
print(theta_train.shape, theta_test.shape)

Pk_train = Pk[:train_val_split]
Pk_test = Pk[train_val_split:]

mask = np.all(Pk_train>0, axis=0)
Pk_train = Pk_train[:,mask]
Pk_test = Pk_test[:,mask]
k = kk[mask]

In [ ]:
# ## ONE TRAINING SAMPLE
# Pk_train = Pk_train[:1]
# theta_train = theta_train[:1]
# print(Pk_train.shape)
# print(theta_train.shape)

In [ ]:
fig, ax = mpl.pyplot.subplots(1,1, figsize=(7,5))
fontsize = 24
fontsize1 = 18

alpha = 1

tmp_Pk_plot = Pk_train
tmp_Pk_plot = tmp_Pk_plot[np.random.choice(tmp_Pk_plot.shape[0], tmp_Pk_plot.shape[0], replace=False)].T
ax.plot(np.log10(k), np.log10(tmp_Pk_plot), c='royalblue', alpha=alpha, lw=0.5)

tmp_Pk_plot = Pk_test
tmp_Pk_plot = tmp_Pk_plot[np.random.choice(tmp_Pk_plot.shape[0], tmp_Pk_plot.shape[0], replace=False)].T
ax.plot(np.log10(k), np.log10(tmp_Pk_plot), c='k', alpha=alpha, lw=0.5)
    
mpl.pyplot.tight_layout()
mpl.pyplot.show()

In [ ]:
dict_bounds = {
    'omega_m'    :  [np.min(theta, axis=0)[0], np.max(theta, axis=0)[0]],
    'omega_b'    :  [np.min(theta, axis=0)[1], np.max(theta, axis=0)[1]],
    'hubble'     :  [np.min(theta, axis=0)[2], np.max(theta, axis=0)[2]],
    'ns'         :  [np.min(theta, axis=0)[3], np.max(theta, axis=0)[3]],
    'sigma8'     :  [np.min(theta, axis=0)[4], np.max(theta, axis=0)[4]],
}

In [ ]:
class Scaler:

    def __init__(self):
          pass
        
    def fit(self, x_train):
        self.x_train_min = np.min(x_train)
        self.x_train_max = np.max(x_train)
           
    def scale(self, x):
        log_x = np.log10(x)
        log_x_norm = (log_x - np.log10(self.x_train_min)) / (np.log10(self.x_train_max) - np.log10(self.x_train_min))
        return log_x_norm
    
    def unscale(self, x_scaled):
        x = x_scaled * (np.log10(self.x_train_max) - np.log10(self.x_train_min)) + np.log10(self.x_train_min)
        return 10**x  

In [ ]:
scaler = Scaler()
scaler.fit(Pk_train)
Pk_train_scaled = scaler.scale(Pk_train)
Pk_test_scaled = scaler.scale(Pk_test)

In [ ]:
print(np.min(Pk_train), np.max(Pk_train))
print(np.min(Pk_train_scaled), np.max(Pk_train_scaled))

print(np.min(Pk_test), np.max(Pk_test))
print(np.min(Pk_test_scaled), np.max(Pk_test_scaled))

In [ ]:
print(Pk_train.shape)
print(theta_train.shape)

In [ ]:
inference, posterior = sbi_tools.train_model(
    theta_train,
    Pk_train_scaled,
    prior= sbi_tools.get_prior(dict_bounds),
    training_batch_size=16,
    validation_fraction=0.2
)

In [ ]:
theta_train_check = theta_train[:1]
Pk_train_scaled_check = Pk_train_scaled[:1]

In [ ]:
# ------------------ posterior inference ------------------ #

# norm_xx_test = scaler.transform(xx_test)

inferred_theta_train_check = sbi_tools.sample_posteriors_theta_test(
    posterior,
    Pk_train_scaled_check,
    dict_bounds,
    N_samples=1000
)

# ------------------ rank stats ------------------ #

ranks_train_check = sbi_tools.compute_ranks(theta_train_check, inferred_theta_train_check)

In [ ]:
N_examples = 1

custom_titles = [
    r'$\Omega_\mathrm{m}$',
    r'$\Omega_\mathrm{b}$',
    r'$h$',
    r'$n_\mathrm{s}$',
    r'$\sigma_{8}$'
]

colors = plot_utils.get_N_colors(N_examples, mpl.colormaps['prism'])
for ii_sample in range(N_examples):
    fig, axs = plot_utils.corner_plot(
        theta_train_check[ii_sample],
        inferred_theta_train_check[ii_sample],
        custom_titles,
        dict_bounds,
        color_infer=colors[ii_sample]
    )
    mpl.pyplot.show()
    
#fig.save("popopo.png")

In [ ]:
# posterior_sample = posterior.sample((1,))
# print(posterior_sample)

In [ ]:
# ------------------ posterior inference ------------------ #

# norm_xx_test = scaler.transform(xx_test)

inferred_theta_test = sbi_tools.sample_posteriors_theta_test(
    posterior,
    Pk_test_scaled,
    dict_bounds,
    N_samples=1000
)

# ------------------ rank stats ------------------ #

ranks = sbi_tools.compute_ranks(theta_test, inferred_theta_test)

In [ ]:
# tmp_inferred_xx_test = sbi_data_utils.compute_baccoemu_predictions_batch(
#     inferred_theta_train_check[indexes],
#     list(dict_bounds.keys())
# )

# fig, ax, ax_res = plot_utils.plot_xx_from_sampled_posteriors(xx_test[indexes], tmp_inferred_xx_test, kk)
# plt.show()

In [ ]:
N_examples = 2

custom_titles = [
    r'$\Omega_\mathrm{m}$',
    r'$\Omega_\mathrm{b}$',
    r'$h$',
    r'$n_\mathrm{s}$',
    r'$\sigma_{8}$'
]

colors = plot_utils.get_N_colors(N_examples, mpl.colormaps['prism'])
for ii_sample in range(N_examples):
    fig, axs = plot_utils.corner_plot(
        theta_test[ii_sample],
        inferred_theta_test[ii_sample],
        custom_titles,
        dict_bounds,
        color_infer=colors[ii_sample]
    )
    mpl.pyplot.show()
    
#fig.save("popopo.png")

In [ ]:
fig, axs = plot_utils.plot_parameter_prediction_vs_truth(inferred_theta_test, theta_test, custom_titles)
plt.tight_layout()
plt.subplots_adjust(wspace=0.6)
plt.show()

In [ ]:
fig, axs = plot_utils.plot_rank_statistcis(ranks, inferred_theta_test.shape[1], custom_titles)
plt.tight_layout()
plt.subplots_adjust(wspace=0.05)
plt.show()